In [1]:
import numpy as np
import os
from sklearn import metrics
#from sklearn.linear_model import LogisticRegression

In [2]:
def load_data(folder):
    x_train = np.load(os.path.join(folder, 'x_train.npy'))
    y_train = np.load(os.path.join(folder, 'y_train.npy'))    
    x_test = np.load(os.path.join(folder, 'x_test.npy'))    
    y_test = np.load(os.path.join(folder, 'y_test.npy'))    
    return x_train, y_train, x_test, y_test


def sigmoid(x):
    return 1 / (1 + np.exp(-x))


class LogisticRegression:
    def __init__(self, dim=2):
        # dim - размерность входных данных.
        self.w = np.random.randn(dim, 1) / np.sqrt(dim) #w = weights
        self.b = np.zeros((1,)) #b = bias
        
    def predict(self, x, probs=False):
        x = x.dot(self.w) + self.b  # logits
        p = sigmoid(x)  # probabilities
        if probs:
            return p
        return np.array(p > 0.5).astype('int32')
        
    def fit(self, x, y, iters=100, lr=0.01):
        # x - np.array размерности [N, dim]
        #     Массив входных признаков.
        # y - np.array размернсоти [N]
        #     Массив меток (правильных ответов).
        # Алгоритм градиентного спуска.
        # Минимизируется бинарная кросс-энтропия.
        y = y.reshape(-1, 1)
        for i in range(iters):
            preds = self.predict(x, probs=True)
            self.w -= lr * np.mean(x.T.dot(preds - y), axis=1, keepdims=True)
            self.b -= lr * np.mean(preds - y, axis=0)

## 1. Применение логистической регрессии (несбалансированные данные)

### 1.1 Создание и обучение логистической регрессии

In [3]:
# Указание: производить нормализацию данных не нужно, это часть задания.
x_train, y_train, x_test, y_test = load_data('dataset1')

In [4]:
# Создайте модель логистической регрессии и обучите её, используя метод fit.
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(x_train, y_train)

y_pred = logreg.predict(x_test)

In [5]:
# Получите предсказания на тестовой выборке и оцените точность модели, 
# используя accuracy_score из пакета SciKit-Learn.
print("Точность модели = ", 
      "{:.0%}".format(
                      metrics.accuracy_score(y_test, y_pred)
                      )
     )

Точность модели =  96%


### 1.2 Анализ качества модели

In [6]:
# Допишите класс "глупого классификатора", что всегда предсказывает класс `0`. 

class DummyClassifier:
    def __init__(self):
        print('DummyClassifier is ready to work!')
        
    def predict(self, x):
        # x - numpy массив размерности [N, dim]
        # Должен возвращаться массив N предсказаний
        N = x.shape[0]
        return np.zeros(N)
    
        pass # если у фукнции или метода нет тела, то нужно писать pass, чтобы код скомпилировался

In [7]:
# Оцените точность "глупого классификатора", объясните результат.
lg_stupid = DummyClassifier()
pred_stupid = lg_stupid.predict(x_test)
print("DummyClassifier's accuracy = ", metrics.accuracy_score(y_test, pred_stupid))

DummyClassifier is ready to work!
DummyClassifier's accuracy =  0.9090909090909091


In [8]:
# Используйте дополнительные метрики из пакета sklearn для анализа "глупого классификатора".
# кроме accuracy, есть другие метрики анализа эффективности модели: recall, precision, f1
lg_stupid_recall = metrics.recall_score(y_test, pred_stupid)
lg_stupid_precision = metrics.precision_score(y_test, pred_stupid)
lg_stupid_f1 = metrics.f1_score(y_test, pred_stupid)

print(" DummyClassifier's recall =", lg_stupid_recall,
     "\n DummyClassifier's precision = ", lg_stupid_precision,
     "\n DummyClassifier's F1 = ", lg_stupid_f1)

 DummyClassifier's recall = 0.0 
 DummyClassifier's precision =  0.0 
 DummyClassifier's F1 =  0.0


C:\Users\1\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Вывод: с DummyClassifier какие-то проблемы. Метрика Accuracy = 0,91. Более точные метрики дают 0. Скорее всего, 90% данных — это нули. Возможно, такая ситуация возникла из-за дисбаланса выборки, то есть из-за ситуации, когда в выборке экземпляторов класса 0 на порядок больше, чем экземляров класса 1.

In [9]:
'''
На протяжении этой лабораторной необходимо оценивать модели по 4 метрикам: accuracy, recall, precision, F1. 
Поэтому в целях упрощения кода и уменьшения его размера имеет смысл написать фукцию, которая будет принимать 
на вход y_test и y_predicted, 
на вывод выдавать значения четырех метрик.
'''

def main_metrics(y_test, y_pred):
    model_accuracy = metrics.accuracy_score(y_test, y_pred)
    model_recall = metrics.recall_score(y_test, y_pred)
    model_precision = metrics.precision_score(y_test, y_pred)
    model_f1 = metrics.f1_score(y_test, y_pred)
    
    print("\n accuracy =",'{:.2f}'.format(model_accuracy),
          "\n recall =", '{:.2f}'.format(model_recall),
         "\n precision = ", '{:.2f}'.format(model_precision),
         "\n F1 = ", '{:.2f}'.format(model_f1)
         )

In [10]:
# Используя те же метрики, проанализируйте модель логистической регрессии. Объясните результат.
print("DummyClassifier's metrics:")
main_metrics(y_test, y_pred)

DummyClassifier's metrics:

 accuracy = 0.96 
 recall = 0.70 
 precision =  0.82 
 F1 =  0.76


## Вывод: модель логистической регрессии работает лучше, чем DummyClassifier.

### 1.3 Анализ набора данных

In [11]:
# Посчитайте количество экземпляров данных для каждого класса.
print(" Кол-во экземпляров класса 0 = ",  sum(y_train == 0), 
      "\n Кол-во экземпляров класса 1 = ", sum(y_train == 1)
     )

 Кол-во экземпляров класса 0 =  200 
 Кол-во экземпляров класса 1 =  20


In [12]:
# Предложите способ улучшения качества модели. Подсказка: добавление дубликатов в данные.
x_train_balanced = x_train[199:219]
y_train_balanced = y_train[199:219]
x_train_new = x_train
y_train_new = y_train

# Создайте и обучите модель с использованием предложенных наработок.
for i in range(0,9):
    x_train_new = np.concatenate((x_train_new, x_train_balanced))
    y_train_new = np.concatenate((y_train_new, y_train_balanced))

lg_balanced = LogisticRegression()
lg_balanced.fit(x_train_new, y_train_new)
lg_balanced_predict = lg_balanced.predict(x_test)

# Оцените качество новой модели, используя метрики из пакета sklearn.metrics. 
print("lg_balanced metrics:")
main_metrics(y_test, lg_balanced_predict)

lg_balanced metrics:

 accuracy = 0.96 
 recall = 0.75 
 precision =  0.83 
 F1 =  0.79


## 2. Применение логистической регрессии (нелинейные данные)

In [13]:
x_train, y_train, x_test, y_test = load_data('dataset2')

In [14]:
# Создайте и обучите модель но этом наборе данных.
class LogisticRegression:
    def __init__(self, dim=2):
        self.w = np.random.randn(dim, 1) / np.sqrt(dim)
        self.b = np.zeros((1,))
        
    def predict(self, x, probs=False):
        x = x.dot(self.w) + self.b
        p = sigmoid(x)
        if probs:
            return p
        return np.array(p > 0.5).astype('int32')
        
    def fit(self, x, y, iters=100, lr=0.01):
        y = y.reshape(-1, 1)
        for i in range(iters):
            preds = self.predict(x, probs=True)
            self.w -= lr * np.mean(x.T.dot(preds - y), axis=1, keepdims=True)
            self.b -= lr * np.mean(preds - y, axis=0)
lg = LogisticRegression()
lg.fit(x_train, y_train)

In [15]:
# Проанализируйте качество модели.
y_pred = lg.predict(x_test)

print("Non-linear model's metrics:")
main_metrics(y_test, y_pred)

Non-linear model's metrics:

 accuracy = 0.48 
 recall = 0.98 
 precision =  0.46 
 F1 =  0.63


In [23]:
# FEATURE ENGINEERING: попробуйте применить на исходных данных разные нелинейные функции (sin, tanh, x^2, ...).
# Объедините трансформированные данные с исходными.
x_train_new = np.concatenate((x_train, np.sin(x_train)), axis=1)
#x_train_new = np.concatenate((x_train, x_train**2), axis=1)

x_test_new = np.concatenate((x_test, np.sin(x_test)), axis=1)

<span style="color:red"> Если делать FEATURE ENGINEERING, то нужно добавлять новые признаки в `x_train`. Но нужно добавлять именно признаки `axis = 1`, но не эксзепляры `axis = 0`. Также если я сделать FEATURE ENGINEERING на тестовых данных `x_train`, то я должен также расширить пространство признаков для `x-test`, то есть сделать те же самые преобразования, которые я делал для `x-train`.</span>

![help_image](help_from_mentor201022.png)

In [24]:
# Создайте и обучите модель с использованием наработок.
lg_new = LogisticRegression(x_train_new.shape[1])
lg_new.fit(x_train_new, y_train)

y_pred_new = lg_new.predict(x_test_new)

In [25]:
# Оцените качество новой модели, используя метрики из пакета sklearn.metrics. 
main_metrics(y_test, y_pred_new)


 accuracy = 0.86 
 recall = 0.69 
 precision =  1.00 
 F1 =  0.82


## 3. Доп. задания (любое на выбор, опционально)

### 3.1 'Упрощение' логистической регрессии

Сложность: легко.

In [ ]:
# Модифицируйте класс логистической регрессии так, чтобы в нём не использовалась сигмоида.
# То есть вывод о предсказанном классе должен делаться на основе значений "до сигмоиды".
# Вспомогательная ссылка: https://en.wikipedia.org/wiki/Logit
# Подсказка: взгляните на то, при каких входных `x` значение сигмоды больше 0.5 и меньше 0.5.

class LogisticRegression:
    def __init__(self, dim=2):
        self.w = np.random.randn(dim, 1) / np.sqrt(dim)
        self.b = np.zeros((1,))
        
    def predict(self, x, probs=False):
        x = x.dot(self.w) + self.b
        p = sigmoid(x)
        if probs:
            return p
        return np.array(p > 0.5).astype('int32')
        
    def fit(self, x, y, iters=100, lr=0.01):
        y = y.reshape(-1, 1)
        for i in range(iters):
            preds = self.predict(x, probs=True)
            self.w -= lr * np.mean(x.T.dot(preds - y), axis=1, keepdims=True)
            self.b -= lr * np.mean(preds - y, axis=0)

In [ ]:
# Повторите эксперимент из задания 1.
x_train, y_train, x_test, y_test = load_data('dataset1')
lg_3 = LogisticRegression(x_train.shape[1])
lg_3.fit(x_train, y_train)

y_pred_3 = lg_3.predict(x_test)

print("Easy Log Regression model's metrics:")
main_metrics(y_test, y_pred_3)

### 3.2 'Обобщение' логистической регрессии

Напишите многоклассовый классификатор. Обучите его на наборе данных ниже.

In [ ]:
x_train, y_train, x_test, y_test = load_data('dataset3')

<b>Ансамбль логистических регрессий.</b> Сложность: супергерой.

In [ ]:
"""
Напишите класс, что инкапсулирует в себе `C` логистических регрессий, 
где `C` - количество классов. i-ая логистическая регрессия производит 
бинарную классификацию вида: все остальные классы и i-ый класс.
"""

class MulticlassLogisticRegression:
    def __init__(self, n_classes, dim):
        pass
    
    def predict(self, x):
        # x - numpy массив размерности [N, dim]
        # Возвращается массив целых чисел размерности [N],
        # где i-ый элемент обозначает номер класса для 
        # i-го экземпляра данных в `x`.
        pass
    
    def fit(self, x, y):
        pass

In [ ]:
# Создайте и обучите написанный классификатор. Оцените точность модели.


<b>Softmax классификатор.</b> Сложность: математический гений.

In [ ]:
"""
Напишите класс классификатора, основанного на функции Softmax.
Алгоритм работы данного классификатора:
x - вектор (экземпляр данных) размерности dim.
W - матрица весов размерности [dim, n_classes].

Ответ классификатора формируется как:
logits = x * W - матричное умножение
p = softmax(logits)
class_id = argmax(p)

Для данного классификатора требуется модифицировать алгоритм обучения в методе fit.

Вспомогательные ресурсы:
https://en.wikipedia.org/wiki/Softmax_function
https://eli.thegreenplace.net/2016/the-softmax-function-and-its-derivative/
"""

class SoftmaxClassificator:
    def __init__(self, n_classes, dim):
        pass
    
    def predict(self, x):
        # x - numpy массив размерности [N, dim]
        # Возвращается массив целых чисел размерности [N],
        # где i-ый элемент обозначает номер класса для 
        # i-го экземпляра данных в `x`.
        pass
    
    def fit(self, x, y):
        pass

In [ ]:
# Создайте и обучите написанный классификатор. Оцените точность модели.


In [ ]:
# Создайте и обучите написанный классификатор на наборе данных из задания 1 (опционально). Оцените точность модели.
